# Chapter 18: Advanced Patterns

Design patterns, descriptors, data classes, and performance optimization



### Descriptors - Property-like Objects (Slide 74)


In [1]:
# Descriptor protocol
class Validator:
    def __init__(self, min_value):
        self.min_value = min_value

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        return obj.__dict__.get(self.name)

    def __set__(self, obj, value):
        if value < self.min_value:
            raise ValueError(f"{self.name} must be >= {self.min_value}")
        obj.__dict__[self.name] = value

class Person:
    age = Validator(0)
    salary = Validator(0)

    def __init__(self, age, salary):
        self.age = age
        self.salary = salary

person = Person(25, 50000)
print(person.age)  # 25

# Validation happens automatically!
try:
    person.age = -5
except ValueError as e:
    print(e)  # age must be >= 0


25
age must be >= 0


> **Note:** Descriptors provide attribute-level control


### Data Classes - Simplified Classes (Slide 75)


In [2]:
from dataclasses import dataclass, field
from typing import List

# Without dataclass - lots of boilerplate
class PersonOld:
    def __init__(self, name, age, hobbies=None):
        self.name = name
        self.age = age
        self.hobbies = hobbies or []

    def __repr__(self):
        return f"Person(name={self.name}, age={self.age})"

    def __eq__(self, other):
        return (self.name, self.age) == (other.name, other.age)

# With dataclass - automatic!
@dataclass
class Person:
    name: str
    age: int
    hobbies: List[str] = field(default_factory=list)

person = Person("Alice", 25, ["coding"])
print(person)  # Person(name='Alice', age=25, hobbies=['coding'])

# Automatic __eq__
person2 = Person("Alice", 25)
print(person == person2)  # False (different hobbies)

# Automatic __repr__, __init__, __eq__, __hash__!


Person(name='Alice', age=25, hobbies=['coding'])
False


> **Note:** dataclass auto-generates boilerplate methods


### __slots__ - Memory Optimization (Slide 76)


In [3]:
import sys

# Regular class - uses __dict__
class PersonDict:
    def __init__(self, name, age):
        self.name = name
        self.age = age

# With __slots__ - no __dict__
class PersonSlots:
    __slots__ = ['name', 'age']

    def __init__(self, name, age):
        self.name = name
        self.age = age

# Memory comparison
p1 = PersonDict("Alice", 25)
p2 = PersonSlots("Alice", 25)

print(f"With dict: {sys.getsizeof(p1.__dict__)} bytes")
print(f"With slots: {sys.getsizeof(p2)} bytes")
# Slots uses ~50% less memory!

# Can't add new attributes to slots
try:
    p2.email = "alice@example.com"
except AttributeError as e:
    print(e)  # No attribute 'email'

# Use __slots__ for:
# - Memory-intensive applications
# - Classes with many instances
# - Fixed set of attributes


With dict: 296 bytes
With slots: 48 bytes
'PersonSlots' object has no attribute 'email' and no __dict__ for setting new attributes


> **Note:** __slots__ reduces memory usage significantly


### Singleton Pattern (Slide 77)


In [4]:
# Method 1: Using decorator
def singleton(cls):
    instances = {}
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return get_instance

@singleton
class Database:
    def __init__(self):
        print("Creating DB connection")

db1 = Database()  # Creating DB connection
db2 = Database()  # No output
print(db1 is db2)  # True

# Method 2: Using __new__
class Singleton:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

class Logger(Singleton):
    def log(self, msg):
        print(f"[LOG] {msg}")

logger1 = Logger()
logger2 = Logger()
print(logger1 is logger2)  # True


Creating DB connection
True
True


> **Note:** Multiple ways to implement Singleton


### Factory Pattern (Slide 78)


In [5]:
# Factory pattern - create objects without specifying class
class Animal:
    def speak(self):
        pass

class Dog(Animal):
    def speak(self):
        return "Woof!"

class Cat(Animal):
    def speak(self):
        return "Meow!"

class AnimalFactory:
    @staticmethod
    def create_animal(animal_type):
        animals = {
            'dog': Dog,
            'cat': Cat
        }

        animal_class = animals.get(animal_type.lower())
        if not animal_class:
            raise ValueError(f"Unknown animal: {animal_type}")

        return animal_class()

# Use factory
animal = AnimalFactory.create_animal('dog')
print(animal.speak())  # Woof!

# Easier to extend - just add to dict!
class Bird(Animal):
    def speak(self):
        return "Tweet!"

# Register new animal type
# AnimalFactory.create_animal.__globals__['animals']['bird'] = Bird


Woof!


> **Note:** Factory pattern for object creation


### Observer Pattern (Slide 79)


In [6]:
# Observer pattern - publish/subscribe
class Observable:
    def __init__(self):
        self._observers = []

    def attach(self, observer):
        self._observers.append(observer)

    def detach(self, observer):
        self._observers.remove(observer)

    def notify(self, *args, **kwargs):
        for observer in self._observers:
            observer.update(*args, **kwargs)

class NewsAgency(Observable):
    def __init__(self):
        super().__init__()
        self._news = None

    def publish_news(self, news):
        self._news = news
        self.notify(news)

class NewsChannel:
    def __init__(self, name):
        self.name = name

    def update(self, news):
        print(f"{self.name} received: {news}")

# Usage
agency = NewsAgency()
channel1 = NewsChannel("BBC")
channel2 = NewsChannel("CNN")

agency.attach(channel1)
agency.attach(channel2)

agency.publish_news("Breaking news!")
# BBC received: Breaking news!
# CNN received: Breaking news!


BBC received: Breaking news!
CNN received: Breaking news!


> **Note:** Observer for event-driven systems


### Context Manager as Decorator (Slide 80)


In [7]:
from contextlib import contextmanager
import time

# Reusable context manager
@contextmanager
def timer(name):
    start = time.time()
    yield
    end = time.time()
    print(f"{name}: {end-start:.4f}s")

# Use as context manager
with timer("Loop"):
    sum(range(1000000))

# Create decorator from context manager
def time_it(func):
    def wrapper(*args, **kwargs):
        with timer(func.__name__):
            return func(*args, **kwargs)
    return wrapper

@time_it
def slow_function():
    time.sleep(1)
    return "Done"

result = slow_function()
# slow_function: 1.0001s

# Combine patterns for maximum flexibility!


Loop: 0.0073s


slow_function: 1.0003s


> **Note:** Context managers complement decorators


### Memoization Pattern (Slide 81)


In [8]:
from functools import wraps

# Custom memoization
def memoize(func):
    cache = {}
    @wraps(func)
    def wrapper(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    wrapper.cache = cache  # Expose cache
    return wrapper

@memoize
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

# Fast!
print(fibonacci(100))  # Instant!
print(len(fibonacci.cache))  # 101 cached values

# Built-in alternative
from functools import lru_cache

@lru_cache(maxsize=128)
def fib_lru(n):
    if n < 2:
        return n
    return fib_lru(n-1) + fib_lru(n-2)

print(fib_lru(100))
print(fib_lru.cache_info())
# CacheInfo(hits=98, misses=101, maxsize=128, currsize=101)


354224848179261915075
101
354224848179261915075
CacheInfo(hits=98, misses=101, maxsize=128, currsize=101)


> **Note:** Memoization for expensive computations


### Method Chaining Pattern (Slide 82)


In [9]:
# Fluent interface - method chaining
class QueryBuilder:
    def __init__(self):
        self._query = {}

    def select(self, *fields):
        self._query['fields'] = fields
        return self  # Return self for chaining

    def from_table(self, table):
        self._query['table'] = table
        return self

    def where(self, condition):
        self._query['where'] = condition
        return self

    def limit(self, n):
        self._query['limit'] = n
        return self

    def build(self):
        return self._query

# Fluent, readable API
query = (QueryBuilder()
    .select('name', 'age')
    .from_table('users')
    .where('age > 18')
    .limit(10)
    .build())

print(query)
# {'fields': ('name', 'age'), 'table': 'users', 
#  'where': 'age > 18', 'limit': 10}

# Common in ORMs, query builders, builders


{'fields': ('name', 'age'), 'table': 'users', 'where': 'age > 18', 'limit': 10}


> **Note:** Return self to enable chaining


### Advanced Python - Key Takeaways (Slide 83)


<p><strong>Patterns Covered:</strong></p>
<ul>
<li><strong>Decorators</strong> - Modify function/class behavior</li>
<li><strong>Generators</strong> - Memory-efficient iteration</li>
<li><strong>Context Managers</strong> - Resource management</li>
<li><strong>Async/Await</strong> - Concurrent I/O operations</li>
<li><strong>Metaclasses</strong> - Control class creation (use sparingly!)</li>
<li><strong>Design Patterns</strong> - Reusable solutions</li>
</ul>
<p><strong>Performance Tips:</strong></p>
<ul>
<li>Use <code>__slots__</code> for memory optimization</li>
<li>Use generators for large datasets</li>
<li>Cache expensive computations</li>
<li>Use async for I/O-bound operations</li>
<li>Profile before optimizing!</li>
</ul>
<p><strong>Best Practices:</strong></p>
<ul>
<li>Simplicity > Cleverness</li>
<li>Use patterns when they solve real problems</li>
<li>Document advanced features well</li>
<li>Test thoroughly</li>
</ul>
